## Création mail Yopmail

In [1]:
import yopmail

email = yopmail.create_random_email()

if email['status'] == 'success':
    generated_email = email['generated_email']
    print(generated_email)
else:
    print(email['error_message'])
    print(email['execution_time'])

seummissiquafre-7681@yopmail.com


## Connexion Hightest

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_argument(f"--window-size={1400},{800}")

driver = webdriver.Chrome(options=options)
driver.get("https://hightest.nc")

# Attendre et cliquer sur Toolbox
toolbox_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//a[@title="Toolbox"]')))
toolbox_link.click()


## CHANGEMENT DE PAGE => https://hightest.nc/toolbox/ ##


# Move to this element 
ISTQB_TITLE_PATH = '//*[@id="page"]/div[2]/div/section/article/div[2]/div/div[1]/div'
button_french = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, ISTQB_TITLE_PATH)))
driver.execute_script("arguments[0].scrollIntoView();", button_french)

# Wait for the element to be clickable
FRENCH_BUTTON_PATH = '//*[@id="page"]/div[2]/div/section/article/div[2]/div/div[1]/div/div[2]/div/a[1]'
toolbox_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, FRENCH_BUTTON_PATH)))
toolbox_link.click()


## CHANGEMENT DE PAGE => https://hightest.nc/ressources/test-istqb.php ##


if driver.current_url != 'https://hightest.nc/ressources/test-istqb.php':
    onglets = driver.window_handles
    driver.switch_to.window(onglets[1])

# Wait submit appear
submit_button = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="submit"]')))

dict_anwsers = {
    1: 1, 2: 2, 3: 1, 4: 2, 5: 2,
    6: 3, 7: 2, 8: 4, 9: 1, 10: 3,
    11: 4, 12: 2, 13: 3, 14: 2, 15: 4,
    16: 3, 17: 3, 18: 1, 19: 2, 20: 2
}

for question in dict_anwsers:
    anwser = dict_anwsers[question]

    # Paths
    MULTI_ANWSERS_PATH = f'//*[@id="main_content"]/form/div[{question}]'
    RADIO_BUTTON_PATH = f'//*[@id="main_content"]/form/div[{question}]/input[{anwser}]'

    # Move to multiple answers.
    question_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, MULTI_ANWSERS_PATH)))
    driver.execute_script("arguments[0].scrollIntoView();", question_element)

    # Click the radio button.
    radio_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, RADIO_BUTTON_PATH)))
    radio_button.click()

# Click the radio button.
SUBMIT_PATH = f'//*[@id="submit"]' 
radio_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, SUBMIT_PATH)))
radio_button.click()


## CHANGEMENT DE PAGE => https://hightest.nc/ressources/reponses-test-istqb.php ##

TEXT_FIELD_PATH = '//*[@id="email"]'
SUBMIT_MAIL_PATH = '//*[@id="submitMail"]'

text_field = driver.find_element(By.XPATH, TEXT_FIELD_PATH)
text_field.send_keys(generated_email)

submit_mail = driver.find_element(By.XPATH, SUBMIT_MAIL_PATH)
submit_mail.click()

driver.quit()

# Let the server send the mail. 
time.sleep(2)

## Lecture mail Yopmail

In [3]:
report = yopmail.last_message_of(email['identifier_email'])

# report = yopmail.last_message_of('zeugreretega-3236')

if report['status'] == 'success':
    content_report = report['content']
    print(content_report)
else:
    print(report['error_message'])
    print(report['execution_time'])

Résultats du test en ligne - Hightest
Vous avez bien répondu à 20 question(s) sur 20, soit 100 % de réussite. Félicitations, vous avez obtenu le score maximal !

Qu'est-ce que le débogage ?

1. Le débogage est l'activité de développement qui consiste à trouver, analyser et corriger les défauts.
2. Le débogage est l'activité de développement qui consiste à vérifier qu'un défaut a bien été corrigé, c'est-à-dire qu'il ne provoque plus de défaillances.
3. Le débogage est l'activité de test qui consiste à rechercher les défaillances et à les compiler dans un outil de test dédié.
4. Le débogage est l'activité de test qui consiste à trier les anomalies des faux positifs.

Solution
Voir chapitre 1.1.2, p. 14.
Vous avez trouvé la bonne réponse !

Lors de la phase de conception des spécifications, quelle pratique est la plus adéquate ?

1. Le testeur n’est pas impliqué dans cette phase
2. Le testeur effectue une revue des spécifications afin de détecter au plus tôt les points à éclaircir ou à co